#### Зададим параболу в моментах времени от 0 до 50 (центр будет в точке 25), сделаем рандомное отклонение от -10 до 10. В итоге получим показания счётчика с рандомной ошибкой в каждом моменте.

In [5]:
import random
sz=51

In [6]:
z_par = []
for i in range(sz):
    z_par.append((i-25)*(i-25)+random.random()*20-10+15)

In [7]:
import plotly
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots

In [8]:
par=go.Figure()
par.add_trace(go.Scatter(x= list(range(0, sz)), y=z_par, name="Датчик"))
ideal_par=[]
for i in range(sz):
    ideal_par.append((i-25)*(i-25)+15)
par.add_trace(go.Scatter(x= list(range(0, sz)), y=ideal_par, name="Реальные координаты"))
par.show()

#### Теперь то же самое сделаем с косинусом, поднятым на 3. Но ошибка, само собой, теперь будет меньше. Возьмём от -0,5 до 0,5.

In [9]:
from math import cos

In [10]:
z_cos = []
for i in range(51):
    z_cos.append(cos(i)+random.random()+2.5)

In [11]:
co=go.Figure()
co.add_trace(go.Scatter(x= list(range(0, sz)), y=z_cos, name="Датчик"))
ideal_cos=[]
for i in range(sz):
    ideal_cos.append(cos(i)+3)
co.add_trace(go.Scatter(x= list(range(0, sz)), y=ideal_cos, name="Реальные координаты"))
co.show()

#### Построим такую же конструкцию для более интересных функций. Например six+exp(x/14), поднятая на 5. Ошибку возьмём от -2 до 2.

In [12]:
from math import sin
from math import exp

In [13]:
z_sinexp = []
for i in range(51):
    z_sinexp.append(sin(i)+exp(i/14)+random.random()*4+3)

In [14]:
sinexp=go.Figure()
sinexp.add_trace(go.Scatter(x= list(range(0, sz)), y=z_sinexp, name="Датчик"))
ideal_sinexp=[]
for i in range(sz):
    ideal_sinexp.append(sin(i)+exp(i/14)+5)
sinexp.add_trace(go.Scatter(x= list(range(0, sz)), y=ideal_sinexp, name="Реальные координаты"))
sinexp.show()

#### Теперь ещё более странная функция. Ошибка от -2 до 2.

In [15]:
from math import log

In [16]:
z_weird = []
for i in range(51):
    z_weird.append(7+5*(sin(5*log(i+1)))+random.random()*4-2)

In [17]:
weird=go.Figure()
weird.add_trace(go.Scatter(x= list(range(0, sz)), y=z_weird, name="Датчик"))
ideal_weird=[]
for i in range(sz):
    ideal_weird.append(7+5*(sin(5*log(i+1))))
weird.add_trace(go.Scatter(x= list(range(0, sz)), y=ideal_weird, name="Реальные координаты"))
weird.show()

#### Напоследок возьмём какую-нибудь функцию, напоминающую затухающее движение.

In [18]:
from math import atan, sqrt

In [19]:
z_fading = []
for i in range(51):
    z_fading.append(20-10*atan(sqrt(i))+random.random()*2-1)

In [20]:
fading=go.Figure()
fading.add_trace(go.Scatter(x= list(range(0, sz)), y=z_fading, name="Датчик"))
ideal_fading=[]
for i in range(sz):
    ideal_fading.append(20-10*atan(sqrt(i)))
fading.add_trace(go.Scatter(x= list(range(0, sz)), y=ideal_fading, name="Реальные координаты"))
fading.show()

#### Теперь, когда у нас есть примеры входных данных, займёмся подсчётом всего, что нужно для фильтра Калмана.

#### Считаем ошибки датчика и их среднее арифметическое, а затем квадратную дисперсию.

In [21]:
errors_par=[]
sr_par=0
for i in range(sz):
    tec = ideal_par[i]-z_par[i]
    sr_par+=tec
    errors_par.append(tec)
sr_par/=sz
disp_par=0
for error in errors_par:
    disp_par+=(error-sr_par)**2
disp_par/=sz
disp_par

32.70945858456678

In [22]:
errors_cos=[]
sr_cos=0
for i in range(sz):
    tec = ideal_cos[i]-z_cos[i]
    sr_cos+=tec
    errors_cos.append(tec)
sr_cos/=sz
disp_cos=0
for error in errors_cos:
    disp_cos+=(error-sr_cos)**2
disp_cos/=sz
disp_cos

0.08895005675264238

In [23]:
errors_sinexp=[]
sr_sinexp=0
for i in range(sz):
    tec = ideal_sinexp[i]-z_sinexp[i]
    sr_sinexp+=tec
    errors_sinexp.append(tec)
sr_sinexp/=sz
disp_sinexp=0
for error in errors_sinexp:
    disp_sinexp+=(error-sr_sinexp)**2
disp_sinexp/=sz
disp_sinexp

1.1958700888928206

In [24]:
errors_weird=[]
sr_weird=0
for i in range(sz):
    tec = ideal_weird[i]-z_weird[i]
    sr_weird+=tec
    errors_weird.append(tec)
sr_weird/=sz
disp_weird=0
for error in errors_weird:
    disp_weird+=(error-sr_weird)**2
disp_weird/=sz
disp_weird

1.2928288884212378

In [25]:
errors_fading=[]
sr_fading=0
for i in range(sz):
    tec = ideal_fading[i]-z_fading[i]
    sr_fading+=tec
    errors_fading.append(tec)
sr_fading/=sz
disp_fading=0
for error in errors_fading:
    disp_fading+=(error-sr_fading)**2
disp_fading/=sz
disp_fading

0.30631837145050606

#### На текущий момент у нас почитаны показания датчика и квадрат дисперсии его ошибок. Теперь посчитаем закон u, по которому будет изменяться движение объекта (для i-го - как координата на i-ом шаге минус координата на i-1-ом шаге), но с ошибкой (независимой от предыдущей). Будем брать интервалы ошибок в 2 раза меньше, раньше (так, если было от -10 до 10, сейчас возьмём от -5 до 5). Для этой ошибки также будем считать среднее, чтобы потом найти дисперсию.

In [26]:
u_par=[]
errors_u_par=[]
sr_u_par = 0
u_par.append(0)
for i in range(1, sz):
    er = random.random()*10-5
    u_par.append(ideal_par[i]-ideal_par[i-1]+er)
    sr_u_par += er
    errors_u_par.append(er)
sr_u_par/=(sz-1)
disp_u_par=0
for error in errors_u_par:
    disp_u_par+=(error-sr_u_par)**2
disp_u_par/=(sz-1)
disp_u_par

9.442621679087452

In [27]:
u_cos=[]
errors_u_cos=[]
sr_u_cos = 0
u_cos.append(0)
for i in range(1, sz):
    er = random.random()*0.5-0.25
    u_cos.append(ideal_cos[i]-ideal_cos[i-1]+er)
    sr_u_cos += er
    errors_u_cos.append(er)
sr_u_cos/=(sz-1)
disp_u_cos=0
for error in errors_u_cos:
    disp_u_cos+=(error-sr_u_cos)**2
disp_u_cos/=(sz-1)
disp_u_cos

0.023278119738879194

In [28]:
u_sinexp=[]
errors_u_sinexp=[]
sr_u_sinexp = 0
u_sinexp.append(0)
for i in range(1, sz):
    er = random.random()*2-1
    u_sinexp.append(ideal_sinexp[i]-ideal_sinexp[i-1]+er)
    sr_u_sinexp += er
    errors_u_sinexp.append(er)
sr_u_sinexp/=(sz-1)
disp_u_sinexp=0
for error in errors_u_sinexp:
    disp_u_sinexp+=(error-sr_u_sinexp)**2
disp_u_sinexp/=(sz-1)
disp_u_sinexp

0.34569370995920984

In [29]:
u_weird=[]
errors_u_weird=[]
sr_u_weird = 0
u_weird.append(0)
for i in range(1, sz):
    er = random.random()*2-1
    u_weird.append(ideal_weird[i]-ideal_weird[i-1]+er)
    sr_u_weird += er
    errors_u_weird.append(er)
sr_u_weird/=(sz-1)
disp_u_weird=0
for error in errors_u_weird:
    disp_u_weird+=(error-sr_u_weird)**2
disp_u_weird/=(sz-1)
disp_u_weird

0.3703856002094188

In [30]:
u_fading=[]
errors_u_fading=[]
sr_u_fading = 0
u_fading.append(0)
for i in range(1, sz):
    er = random.random()-0.5
    u_fading.append(ideal_fading[i]-ideal_fading[i-1]+er)
    sr_u_fading += er
    errors_u_fading.append(er)
sr_u_fading/=(sz-1)
disp_u_fading=0
for error in errors_u_fading:
    disp_u_fading+=(error-sr_u_fading)**2
disp_u_fading/=(sz-1)
disp_u_fading

0.0708894810966009

#### Теперь у нас посчитаны законы U и квадраты дисперсии их ошибок. Осталось аккуратно записать все данные.

In [31]:
with open('format.txt', 'w') as f:
    f.write("Количество точек.\n")
    f.write("Квадрат дисперсии ошибок датчика.\n")
    f.write("Квадрат дисперсии ошибок закона.\n")
    f.write(f"{sz} строк - показания датчика.\n")
    f.write(f"{sz} строк - закон измения координат.\n")

In [32]:
with open('par.txt', 'w') as f:
    f.write(f"{sz}\n")
    f.write(f"{disp_par}\n")
    f.write(f"{disp_u_par}\n")
    for i in z_par:
        f.write(f"{i}\n")
    for i in u_par:
        f.write(f"{i}\n")

In [33]:
with open('cos.txt', 'w') as f:
    f.write(f"{sz}\n")
    f.write(f"{disp_cos}\n")
    f.write(f"{disp_u_cos}\n")
    for i in z_cos:
        f.write(f"{i}\n")
    for i in u_cos:
        f.write(f"{i}\n")

In [34]:
with open('sinexp.txt', 'w') as f:
    f.write(f"{sz}\n")
    f.write(f"{disp_sinexp}\n")
    f.write(f"{disp_u_sinexp}\n")
    for i in z_sinexp:
        f.write(f"{i}\n")
    for i in u_sinexp:
        f.write(f"{i}\n")

In [35]:
with open('weird.txt', 'w') as f:
    f.write(f"{sz}\n")
    f.write(f"{disp_weird}\n")
    f.write(f"{disp_u_weird}\n")
    for i in z_weird:
        f.write(f"{i}\n")
    for i in u_weird:
        f.write(f"{i}\n")

In [36]:
with open('fading.txt', 'w') as f:
    f.write(f"{sz}\n")
    f.write(f"{disp_fading}\n")
    f.write(f"{disp_u_fading}\n")
    for i in z_fading:
        f.write(f"{i}\n")
    for i in u_fading:
        f.write(f"{i}\n")